In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
"""Download Imagenet train Dataset"""

def download_data():

    for i in os.listdir():
        
        if '.tar' in i:
            
            os.remove(i)
    
    !wget https://w251hw05.s3.us-west-1.amazonaws.com/ILSVRC2012_img_train.tar

    #!wget https://w251hw05.s3.us-west-1.amazonaws.com/ILSVRC2012_img_val.tar
    
    print(os.listdir())

#download_data()

In [11]:
def extract_data():
    
    import tarfile 

    if os.path.exists('Train'):
        
        print('extracting training data only')
        
        """Extract all .Tar Files in Training Data"""
    
        for i in range(len(os.listdir('Train'))):

            if '.tar' in os.listdir('Train')[i]:

                tftn = tarfile.open(f"Train/{os.listdir('Train')[i]}")

                tftn.extractall(f'Train/{i}')

                tftn.close()

                print(i, len(os.listdir(f'Train/{i}')))


        """Remove all .tar files from directory"""


        for i in os.listdir('Train'):

            if '.tar' in i: 

                os.remove(f'Train/{i}')
        
    else: 
        
        tft = tarfile.open('ILSVRC2012_img_train.tar')

        tft.extractall('Train')

        tft.close()

        if os.path.exists('ILSVRC2012_img_val.tar'):

            tfv = tarfile.open('ILSVRC2012_img_val.tar')

            tfv.extractall('Val')

            tfv.close()

        """Extract all .Tar Files in Training Data"""

        for i in range(len(os.listdir('Train'))):

            if '.tar' in os.listdir('Train')[i]:

                tftn = tarfile.open(f"Train/{os.listdir('Train')[i]}")

                tftn.extractall(f'Train/{i}')

                tftn.close()

                print(i, len(os.listdir(f'Train/{i}')))


        """Remove all .tar files from directory"""


        for i in os.listdir('Train'):

            if '.tar' in i: 

                os.remove(f'Train/{i}')

In [12]:
extract_data()

0 1300
1 1300
2 1300
3 1300
4 1300
5 1300
6 1300
7 1300
8 1300
9 1300
10 1300
11 1300
12 1300
13 755
14 1300
15 1300
16 1300
17 1207
18 1300
19 1300
20 1300
21 1053
22 1055
23 1300
24 1300
25 1300
26 1300
27 1300
28 1300
29 1300
30 1300
32 1300
33 1300
34 1071
35 1300
36 1300
37 1300
38 1300
39 1300
40 1300
41 1300
43 1300
44 1300
45 1300
46 1053
47 1300
48 1300
49 1300
50 1300
51 1300
52 1300
54 1300
55 1300
56 1300
57 1300
58 1300
59 1300
60 1300
61 1300
62 1300
64 1300
65 1300
66 1300
67 1300
68 1300
69 1300
70 889
71 889
72 1300
73 1300
74 1300
75 1300
76 1300
77 954
78 1300
79 1300
80 1300
81 1300
83 1300
84 1300
85 1300
86 1300
88 1300
89 1300
90 1300
91 1300
92 1300
93 1300
94 1300
95 1300
96 1300
98 1300
99 1300
101 1300
102 1084
103 1300
104 1300
106 1300
107 1300
108 1300
109 1300
110 1300
111 1300
112 1300
113 1300
114 1300
115 1300
116 1300
117 1300
118 1300
119 1300
120 1300
121 1300
122 1300
123 1300
125 1300
127 1300
128 1300
129 1300
130 1300
131 1300
132 1300
133 1300


In [9]:
# for i in os.listdir('Train'):
    
#     if os.path.isdir(f'Train/{i}'):
        
#         for j in os.listdir(f'Train/{i}'):
            
#             os.remove(f'Train/{i}/{j}')
            
#         os.rmdir(f'Train/{i}')
            
#     else:
        
#         os.remove(f'Train/{i}')
            
# os.rmdir('Train')

In [ ]:
SEED = 1 
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

torch.cuda.device_count()

START_EPOCH = 0

#Model 

ARCH = 'resnet18'
EPOCHS = 20
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 50
TRAIN_BATCH=128
VAL_BATCH=128
WORKERS=2
TRAINDIR="Train/"
VALDIR="Val/"

device = torch.cuda.set_device('cuda:0')

device = 'GPU'

In [ ]:
"""Use TensorBoard for nice model training viz"""

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [ ]:
def train(train_loader, model, criterion, optimizer, epoch, writer, amp = True):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    
    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if device is not None:
            images = images.cuda(device, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(device, non_blocking=True)

        # compute output, Use Torch Autocast for Automated Mixed Precision
        
        if amp: 
            
            with autocast(dtype=torch.float16): #device_type='cuda', 
                output = model(images)
                loss = criterion(output, target)
                
        else:
            
            output = model(images)
            loss = criterion(output, target)
            

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))
        

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            progress.display(i)
            
#     writer.add_scalar("Loss/train", losses, epoch)
#     writer.add_scalar("acc1/train", top1, epoch)
#     writer.add_scalar("acc5/train", top5, epoch)
#     writer.add_scalar("Time/train", batch_time, epoch)
    
    return [losses, top1, top5, batch_time]

In [ ]:
def validate(val_loader, model, criterion, writer):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    
    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if device is not None:
                images = images.cuda(device, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(device, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

#     writer.add_scalar("Loss/val", losses, epoch)
#     writer.add_scalar("acc1/val", top1, epoch)
#     writer.add_scalar("acc5/val", top5, epoch)
#     writer.add_scalar("Time/val", batch_time, epoch)
    

    return [top1.avg, losses, top1, top5,  batch_time]

In [ ]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [ ]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [ ]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [ ]:
"""Standardize RGB channels"""

imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]

normalize = transforms.Normalize(mean=imagenet_mean_RGB, std= imagenet_std_RGB)



In [ ]:
NUM_CLASSES = 1000

model = models.__dict__[ARCH]()

inf = model.fc.in_features

model.fc = nn.Linear(inf, NUM_CLASSES)

model.cuda('cuda')

In [ ]:
criterion = nn.CrossEntropyLoss().cuda(device)

optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4, pad_if_needed = True), #ran into an error where the image was below the 32,32 image size 
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

TRAINDIR = 'Train/'
print(TRAINDIR)


train_dataset = datasets.ImageFolder(
    TRAINDIR, transform=transform_train)

In [ ]:
"""Split Train and validate on Training Image folders because 
Validate datafolder did not come with lables"""
import numpy as np

orig_set = torchvision.datasets.ImageFolder(TRAINDIR, transform=transform_train) 

n = len(orig_set)  # total number of examples

print(n)

sub = np.array(random.sample(range(n), int(n*0.1))) # Random Subset with seed = 1 for repeatability 

sub_trn = np.arange(n)
sub_trn = sub_trn[~sub]

print(len(sub_trn), len(sub))
print(sub[69]) # prove that the numbers are the same across the two notebooks 

n_test = int(0.1 * n)  # take ~10% for test
val_set = torch.utils.data.Subset(orig_set, sub)  # take first 10%
train_set = torch.utils.data.Subset(orig_set, range(n_test, n))  # take the rest  

In [ ]:
train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=TRAIN_BATCH, shuffle=True,
        num_workers=WORKERS, pin_memory=True, sampler=None)

val_loader = torch.utils.data.DataLoader(
        val_set, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None) 

In [ ]:
best_acc1 = 0

"""upgrade my version of pytorch to get usage of autocast"""

!pip install --upgrade torch

from torch.cuda.amp import autocast



In [ ]:
from torch.cuda.amp import autocast 

resnet_18_single_node_scores_train = [] 
resnet_18_single_node_scores_val = [] 

for epoch in range(START_EPOCH, EPOCHS):
#    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train_scores = train(train_loader, model, criterion, optimizer, epoch, writer)

    print(type(train_scores[0]))
    # evaluate on validation set
    val_scores = validate(val_loader, model, criterion, writer)
    acc1 = val_scores[0]
    
    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))
    
    resnet_18_single_node_scores_train.append(train_scores)
    resnet_18_single_node_scores_val.append(val_scores)

# Okay we have run several models and have collected the data on their performance differences

# Lets move to Distributed Multi Processing 

In [13]:
"""Import Necessary Libraries"""
import torch.multiprocessing as mp

from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

"""Check Backends and GPUs"""

print(torch.distributed.is_available())

print(torch.distributed.is_gloo_available())

print(torch.distributed.is_nccl_available())

print(torch.distributed.is_mpi_available())

print('GPUs on device: ', torch.cuda.device_count())

True
True
True
True
GPUs on device:  1


In [ ]:
init_process_group("gloo", rank=0, world_size=2, init_method='tcp://172.31.87.52:12355')
print('process groups initialized') 

# This is where the process hangs and will not continue. I followed along many Pytorch tutorials, opened all ports on my docker containers, and opened my security groups for all inbound IP traffic from internal IP addresses of my AWS ec2 instances 

In [ ]:
"""Modify Data Loaders for Distributed Sampling"""

def Distrib_sample(train_loader, val_loader, TRAIN_BATCH):
    
    train_loader = torch.utils.data.DataLoader(
            train_set, batch_size=TRAIN_BATCH, shuffle=True,
            num_workers=WORKERS, pin_memory=True, sampler=DistributedSampler(train_set))

    val_loader = torch.utils.data.DataLoader(
            val_set, batch_size=TRAIN_BATCH, shuffle=True,
            num_workers=WORKERS, pin_memory=True, sampler=DistributedSampler(val_set))
    
    return train_loader, val_loader

In [ ]:
def run(ARCH, EPOCHS, BATCH, train_loader, val_loader, rank): 
    
    #Initiate DDP 
    setup(rank, 2)
    torch.cuda.set_device(rank)
    
    """Training Function"""
    LR = 0.1
    MOMENTUM = 0.9
    WEIGHT_DECAY = 1e-4
    PRINT_FREQ = 50
    TRAIN_BATCH=BATCH
    VAL_BATCH= BATCH
    WORKERS=2


    #Create Model with Imagenet output Layer
    criterion = nn.CrossEntropyLoss().cuda(device)
    optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    
    model = models.__dict__[ARCH]()
    inf = model.fc.in_features
    model.fc = nn.Linear(inf, NUM_CLASSES)
    model.cuda('cuda')
    model = DDP(model, device_ids = [0])

    #Instantiate Metrics Lists 
    train_metrics = [] 
    val_metrics = [] 
    
    #wrap Data Loaders in Distributed Sampler 
    train_loader_val_loader = Distrib_sample(train_loader, val_loader, TRAIN_BATCH)
    
    for epoch in range(START_EPOCH, EPOCHS):
        
        #Modify Learning Rate (Cosine Annealing)
        #adjust_learning_rate(optimizer, epoch)

        # train for one epoch
        train_scores = train(train_loader, model, criterion, optimizer, epoch, writer, amp = True)

        print(type(train_scores[0]))
        # evaluate on validation set
        val_scores = validate(val_loader, model, criterion, writer)
        acc1 = val_scores[0]

        # remember best acc@1 and save checkpoint
        is_best = acc1 > best_acc1
        best_acc1 = max(acc1, best_acc1)


        save_checkpoint({
            'epoch': epoch + 1,
            'arch': ARCH,
            'state_dict': model.state_dict(),
            'best_acc1': best_acc1,
            'optimizer' : optimizer.state_dict(),
        }, is_best)

        scheduler.step()
        print('lr: ' + str(scheduler.get_last_lr()))

        #record performance
        prof.step()
        train_metrics.append(train_scores)
        val_metrics.append(val_scores)

    return train_metrics, val_metrics